In [2]:
%matplotlib inline  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import goatools

In [3]:
# extract the relevent tissue types
tissues = {}
inputFilename = '../data/Tissue_Name_Mappings.csv' # manually curated by Nico
inputFile = open(inputFilename)
atLine = 0
for line in inputFile:
    str_list = line.split(',')[1:]
    for term in str_list:
        if term == '' or term == '\r\n':
            continue
        else:
            term = term.split('\r\n')[0] # to avoid end of string in the last term
            tissues[term] = int(term.split('(')[1].split(')')[0])
inputFile.close()

# extract the relevant GO terms
GOMap = {}
inputFilename = '../data/BTO_GO.csv'
inputFile = open(inputFilename)
atLine = 0
skipLines = 2
for line in inputFile:
    if atLine < skipLines:
        atLine += 1
        continue
    str_list = line.split(',')[1:]
    tissue = str_list[0]
    if tissue in tissues:
        for term in str_list[1:]:
            if term == '' or term == '\r\n':
                continue
            else: 
                term = term.split('\r\n')[0] # to avoid end of string in the last term
                if term not in GOMap:
                    GOMap[term] = [tissue]    
                else:
                    GOMap[term].append(tissue)
                    #print tissue, term
inputFile.close()
print 'Number of tissues in BTO : ' + str(len(tissues))
print 'Number of GO terms       : ' + str(len(GOMap))

Number of tissues in BTO : 39
Number of GO terms       : 1641


In [7]:
from goatools.associations import read_ncbi_gene2go
from goatools.base import download_ncbi_associations

def map_entrez_to_ensembl(path):
    dict = {}
    file = open(path)
    for line in file:
        vals = line.split('\t')
        ens_gene_id = vals[0]
        entrez_id = vals[2]
        dict[entrez_id] = ens_gene_id

    file.close()
    return dict

def get_ensembl_ids(go_process_id, entrez_to_ensembl):
    gene2go = '../data/gene2go' # gene2go = download_ncbi_associations()
    # taxids=[9606] means select only human.
    # TODO: ask Marinka if we should use EXP code for evidence!!
    go_to_entrez_ids_human = read_ncbi_gene2go(gene2go, taxids=[9606], go2geneids=True)
    """, evidence_set='EXP'"""

    entrez_ids = go_to_entrez_ids_human[GO_PROCESS_ID]
    # ensembl_ids = []
    # for ent_id in entrez_ids:
    #    ensembl_ids.append(entrez_to_ensembl[str(ent_id)])
    # print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go_to_entrez_ids_human)))
    #return ensembl_ids
    return entrez_ids



In [8]:
geneCounts = []
biomart_file_path = '../data/biomart_ensembl_to_entrez.txt'
path_to_rpkm_file = '../local_data/transcript_rpkm_top_10000_var.txt'
entrez_to_ensembl = map_entrez_to_ensembl(biomart_file_path)
for key in GOMap:
    GO_PROCESS_ID = key  # Biological Process ID in Gene Ontology
    ensembl_ids = get_ensembl_ids(GO_PROCESS_ID, entrez_to_ensembl)
    num_ensemble_ids = len(ensembl_ids)
    geneCounts.append(num_ensemble_ids)
    print key + ': # of Ensemble IDs: ', num_ensemble_ids

#gene_ids_ordered = []
#NUM_SAMPLES = 8555
#gene_features = np.empty((0, NUM_SAMPLES))

16439 GO terms associated with human NCBI Entrez GeneIDs
# of Ensemble IDs:  1
16439 GO terms associated with human NCBI Entrez GeneIDs
# of Ensemble IDs:  1
16439 GO terms associated with human NCBI Entrez GeneIDs
# of Ensemble IDs:  16
16440 GO terms associated with human NCBI Entrez GeneIDs
# of Ensemble IDs:  0
16440 GO terms associated with human NCBI Entrez GeneIDs
# of Ensemble IDs:  0


KeyboardInterrupt: 